In [65]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

Using TensorFlow backend.


In [33]:


prefix = 'v2/'
train_df = pd.read_csv(prefix + 'training_v2.tsv',  sep='\t', header=None)
eval_df = pd.read_csv(prefix + 'dev_v2.tsv', sep='\t', header=None)

train_df.columns = list(train_df.iloc[0])
train_df = train_df[1:]
eval_df.columns = list(eval_df.iloc[0])
eval_df = eval_df[1:]

eval_df.head()

,topic_id,tweet_id,tweet_url,tweet_text,claim,check_worthiness
1,covid-19,1235714275752267776,https://twitter.com/julialindau/status/1235714...,I just landed at JFK after reporting on #coron...,1,1
2,covid-19,1235256530728972290,https://twitter.com/stayfrea_/status/123525653...,ALERT‼️‼️‼️ The corona virus can be spread thr...,1,0
3,covid-19,1235648554338791427,https://twitter.com/A6Asap/status/123564855433...,COVID-19 health advice⚠️ https://t.co/XsSAo52Smu,0,0
4,covid-19,1235674258858061825,https://twitter.com/DrDenaGrayson/status/12356...,⚠️Chinese doctors say autopsies of #coronaviru...,1,1
5,covid-19,1235663306246860800,https://twitter.com/NNUBonnie/status/123566330...,.@NationalNurses President Deborah Burger read...,0,0


In [34]:
print(train_df.shape)
print('\nclaim:\n',train_df.claim.value_counts())
print('\ncheck_worthiness:\n',train_df.check_worthiness.value_counts())

print('\n',eval_df.shape)
print('\nclaim:\n',eval_df.claim.value_counts())
print('\ncheck_worthiness:\n',eval_df.check_worthiness.value_counts())

(672, 6)

claim:
 1    410
0    262
Name: claim, dtype: int64

check_worthiness:
 0    441
1    231
Name: check_worthiness, dtype: int64

 (150, 6)

claim:
 1    97
0    53
Name: claim, dtype: int64

check_worthiness:
 0    91
1    59
Name: check_worthiness, dtype: int64


In [35]:
import re
import string

def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

train_df['text_clean'] = train_df['tweet_text'].apply(lambda x: remove_punct(x))
eval_df['text_clean'] = eval_df['tweet_text'].apply(lambda x: remove_punct(x))
eval_df.head()

,topic_id,tweet_id,tweet_url,tweet_text,claim,check_worthiness,text_clean
1,covid-19,1235714275752267776,https://twitter.com/julialindau/status/1235714...,I just landed at JFK after reporting on #coron...,1,1,I just landed at JFK after reporting on corona...
2,covid-19,1235256530728972290,https://twitter.com/stayfrea_/status/123525653...,ALERT‼️‼️‼️ The corona virus can be spread thr...,1,0,ALERT‼️‼️‼️ The corona virus can be spread thr...
3,covid-19,1235648554338791427,https://twitter.com/A6Asap/status/123564855433...,COVID-19 health advice⚠️ https://t.co/XsSAo52Smu,0,0,COVID19 health advice⚠️ httpstcoXsSAo52Smu
4,covid-19,1235674258858061825,https://twitter.com/DrDenaGrayson/status/12356...,⚠️Chinese doctors say autopsies of #coronaviru...,1,1,⚠️Chinese doctors say autopsies of coronavirus...
5,covid-19,1235663306246860800,https://twitter.com/NNUBonnie/status/123566330...,.@NationalNurses President Deborah Burger read...,0,0,NationalNurses President Deborah Burger reads ...


In [37]:
from nltk import word_tokenize

train_tokens = [word_tokenize(sen) for sen in train_df.text_clean]
eval_tokens = [word_tokenize(sen) for sen in eval_df.text_clean]

def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
train_lower_tokens = [lower_token(token) for token in train_tokens]
eval_lower_tokens = [lower_token(token) for token in eval_tokens]

In [41]:
from nltk.corpus import stopwords

stoplist = stopwords.words('english')

def removeStopWords(tokens): 
    return [word for word in tokens if word not in stoplist]

train_filtered_words = [removeStopWords(sen) for sen in train_lower_tokens]
train_df['Text_Final'] = [' '.join(sen) for sen in train_filtered_words]
train_df['tokens'] = train_filtered_words

eval_filtered_words = [removeStopWords(sen) for sen in eval_lower_tokens]
eval_df['Text_Final'] = [' '.join(sen) for sen in eval_filtered_words]
eval_df['tokens'] = eval_filtered_words

eval_df.head()

,topic_id,tweet_id,tweet_url,tweet_text,claim,check_worthiness,text_clean,Text_Final,tokens
1,covid-19,1235714275752267776,https://twitter.com/julialindau/status/1235714...,I just landed at JFK after reporting on #coron...,1,1,I just landed at JFK after reporting on corona...,landed jfk reporting coronavirus milan lombard...,"[landed, jfk, reporting, coronavirus, milan, l..."
2,covid-19,1235256530728972290,https://twitter.com/stayfrea_/status/123525653...,ALERT‼️‼️‼️ The corona virus can be spread thr...,1,0,ALERT‼️‼️‼️ The corona virus can be spread thr...,alert‼️‼️‼️ corona virus spread money money ho...,"[alert‼️‼️‼️, corona, virus, spread, money, mo..."
3,covid-19,1235648554338791427,https://twitter.com/A6Asap/status/123564855433...,COVID-19 health advice⚠️ https://t.co/XsSAo52Smu,0,0,COVID19 health advice⚠️ httpstcoXsSAo52Smu,covid19 health advice⚠️ httpstcoxssao52smu,"[covid19, health, advice⚠️, httpstcoxssao52smu]"
4,covid-19,1235674258858061825,https://twitter.com/DrDenaGrayson/status/12356...,⚠️Chinese doctors say autopsies of #coronaviru...,1,1,⚠️Chinese doctors say autopsies of coronavirus...,⚠️chinese doctors say autopsies coronavirus vi...,"[⚠️chinese, doctors, say, autopsies, coronavir..."
5,covid-19,1235663306246860800,https://twitter.com/NNUBonnie/status/123566330...,.@NationalNurses President Deborah Burger read...,0,0,NationalNurses President Deborah Burger reads ...,nationalnurses president deborah burger reads ...,"[nationalnurses, president, deborah, burger, r..."


In [48]:
pos = []
neg = []
for l in train_df.claim:
    if int(l) == 0:
        pos.append(0)
        neg.append(1)
    elif int(l) == 1:
        pos.append(1)
        neg.append(0)
        
train_df['claim_Pos']= pos
train_df['claim_Neg']= neg

train_claim_df = train_df[['Text_Final', 'tokens', 'claim', 'claim_Pos', 'claim_Neg']]

pos = []
neg = []
for l in eval_df.claim:
    if int(l) == 0:
        pos.append(0)
        neg.append(1)
    elif int(l) == 1:
        pos.append(1)
        neg.append(0)
        
eval_df['claim_Pos']= pos
eval_df['claim_Neg']= neg

eval_claim_df = eval_df[['Text_Final', 'tokens', 'claim', 'claim_Pos', 'claim_Neg']]

eval_claim_df.head()

,Text_Final,tokens,claim,claim_Pos,claim_Neg
1,landed jfk reporting coronavirus milan lombard...,"[landed, jfk, reporting, coronavirus, milan, l...",1,1,0
2,alert‼️‼️‼️ corona virus spread money money ho...,"[alert‼️‼️‼️, corona, virus, spread, money, mo...",1,1,0
3,covid19 health advice⚠️ httpstcoxssao52smu,"[covid19, health, advice⚠️, httpstcoxssao52smu]",0,0,1
4,⚠️chinese doctors say autopsies coronavirus vi...,"[⚠️chinese, doctors, say, autopsies, coronavir...",1,1,0
5,nationalnurses president deborah burger reads ...,"[nationalnurses, president, deborah, burger, r...",0,0,1


In [49]:
pos = []
neg = []
for l in train_df.check_worthiness:
    if int(l) == 0:
        pos.append(0)
        neg.append(1)
    elif int(l) == 1:
        pos.append(1)
        neg.append(0)
        
train_df['worth_Pos']= pos
train_df['worth_Neg']= neg

train_worth_df = train_df[['Text_Final', 'tokens', 'check_worthiness', 'worth_Pos', 'worth_Neg']]

pos = []
neg = []
for l in eval_df.check_worthiness:
    if int(l) == 0:
        pos.append(0)
        neg.append(1)
    elif int(l) == 1:
        pos.append(1)
        neg.append(0)
        
eval_df['worth_Pos']= pos
eval_df['worth_Neg']= neg

eval_worth_df = eval_df[['Text_Final', 'tokens', 'check_worthiness', 'worth_Pos', 'worth_Neg']]

eval_worth_df.head()

,Text_Final,tokens,check_worthiness,worth_Pos,worth_Neg
1,landed jfk reporting coronavirus milan lombard...,"[landed, jfk, reporting, coronavirus, milan, l...",1,1,0
2,alert‼️‼️‼️ corona virus spread money money ho...,"[alert‼️‼️‼️, corona, virus, spread, money, mo...",0,0,1
3,covid19 health advice⚠️ httpstcoxssao52smu,"[covid19, health, advice⚠️, httpstcoxssao52smu]",0,0,1
4,⚠️chinese doctors say autopsies coronavirus vi...,"[⚠️chinese, doctors, say, autopsies, coronavir...",1,1,0
5,nationalnurses president deborah burger reads ...,"[nationalnurses, president, deborah, burger, r...",0,0,1


In [50]:
all_training_words = [word for tokens in train_worth_df["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in train_worth_df["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))


13190 words total, with a vocabulary size of 4960
Max sentence length is 52


In [51]:
all_test_words = [word for tokens in eval_worth_df["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in eval_worth_df["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

3097 words total, with a vocabulary size of 1673
Max sentence length is 49


In [67]:

MAX_SEQUENCE_LENGTH = 10000
EMBEDDING_DIM = 300

worth_df = pd.concat([eval_worth_df, train_worth_df])

model = models.Word2Vec(sentences = worth_df['tokens'], size=EMBEDDING_DIM, window=5, workers=4, min_count=1)
filename = 'gensim_word2vec_model.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [70]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [71]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [73]:
training_embeddings = get_word2vec_embeddings(word2vec, train_worth_df, generate_missing=True)

MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 300

In [74]:

tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(train_worth_df["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(train_worth_df["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 4960 unique tokens.


In [75]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [76]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(4961, 300)


In [77]:
test_sequences = tokenizer.texts_to_sequences(eval_worth_df["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [78]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [81]:
label_names = ['worth_Pos', 'worth_Neg']

In [82]:
y_train = train_worth_df[label_names].values

In [83]:
x_train = train_cnn_data
y_tr = y_train

In [84]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 300)    1488300     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 999, 200)     120200      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 998, 200)     180200      embedding_1[0][0]                
____________________________________________________________________________________________

In [108]:
num_epochs = 10
batch_size = 32

In [109]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Train on 604 samples, validate on 68 samples
Epoch 1/10
604/604 [==============================] - 37s 61ms/step - loss: 0.3567 - acc: 0.8725 - val_loss: 0.7831 - val_acc: 0.5588
Epoch 2/10
604/604 [==============================] - 35s 59ms/step - loss: 0.2683 - acc: 0.8965 - val_loss: 0.7063 - val_acc: 0.6103
Epoch 3/10
604/604 [==============================] - 39s 64ms/step - loss: 0.1307 - acc: 0.9793 - val_loss: 0.8702 - val_acc: 0.6324
Epoch 4/10
604/604 [==============================] - 38s 63ms/step - loss: 0.0674 - acc: 0.9942 - val_loss: 0.7820 - val_acc: 0.6397
Epoch 5/10
604/604 [==============================] - 38s 63ms/step - loss: 0.0348 - acc: 0.9983 - val_loss: 0.7706 - val_acc: 0.6250
Epoch 6/10
604/604 [==============================] - 38s 63ms/step - loss: 0.0200 - acc: 1.0000 - val_loss: 0.9332 - val_acc: 0.6471
Epoch 7/10
604/604 [==============================] - 38s 63ms/step - loss: 0.0110 - acc: 1.0000 - val_loss: 0.9499 - val_acc: 0.6618
Epoch 8/10
604/60

In [110]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

predictions

150/150 [==============================] - 4s 27ms/step


array([[4.28962708e-02, 9.65939522e-01],
       [2.17623115e-02, 9.86365318e-01],
       [5.19275665e-04, 9.99599457e-01],
       [5.35125136e-02, 9.49980259e-01],
       [3.10888439e-01, 7.36606359e-01],
       [6.75777316e-01, 3.91825020e-01],
       [3.56486440e-03, 9.97589648e-01],
       [7.41099211e-05, 9.99958277e-01],
       [3.54892015e-03, 9.97552156e-01],
       [4.70826030e-03, 9.96361136e-01],
       [5.55048227e-01, 5.15073359e-01],
       [1.93297118e-01, 8.23266447e-01],
       [3.70458722e-01, 6.96191967e-01],
       [8.00428391e-02, 9.28226948e-01],
       [4.28771973e-03, 9.96693969e-01],
       [8.86392236e-01, 1.68640614e-01],
       [1.55294806e-01, 8.56307030e-01],
       [2.10961938e-01, 7.92540193e-01],
       [2.02807784e-02, 9.83283758e-01],
       [1.89784467e-02, 9.80797708e-01],
       [1.64558887e-01, 8.59206319e-01],
       [2.55733728e-04, 9.99836683e-01],
       [4.55227196e-02, 9.61600780e-01],
       [1.12549940e-04, 9.99902666e-01],
       [9.391516

In [111]:
labels = [1, 0]

In [112]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])
    
print(len(prediction_labels))
prediction_labels

#eval_worth_df.check_worthiness

150


[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [113]:
sum(eval_worth_df.check_worthiness==prediction_labels)/len(prediction_labels)
print(eval_worth_df.check_worthiness==prediction_labels)

1      False
2      False
3      False
4      False
5      False
       ...  
146    False
147    False
148    False
149    False
150    False
Name: check_worthiness, Length: 150, dtype: bool


In [114]:
eval_worth_df.check_worthiness.value_counts()

0    91
1    59
Name: check_worthiness, dtype: int64

In [115]:
from sklearn.metrics import classification_report

truth = []
for l in eval_worth_df.check_worthiness:
    truth.append(int(l))


print(classification_report(truth, prediction_labels))

              precision    recall  f1-score   support

           0       0.67      0.96      0.79        91
           1       0.80      0.27      0.41        59

    accuracy                           0.69       150
   macro avg       0.73      0.61      0.60       150
weighted avg       0.72      0.69      0.64       150

